In [9]:
import pandas as pd
import os
import gzip
import json
import zlib
import numpy as np
import sys


import ast


import numpy as np
import importlib

sys.path.append('data/data_utils')
import data_utils
importlib.reload(data_utils)
from data_utils import *

In [1]:

run_id = 514
# file_path= f'/checkpoint/maui_sft/winnieyangwn/amaia_dumps/{run_id}/trajectories/mle_bench_bashmle_bench_checkpoint_maui_sft_shared_kniu_datasets_mlebench_full_jsonl/mle_bench_bashmle_bench_checkpoint_maui_sft_shared_kniu_datasets_mlebench_full_jsonl.jsonl'
file_path = f'/checkpoint/maui_sft/winnieyangwn/amaia_dumps/{run_id}/trajectories/mle_bench_bashmle_bench_checkpoint_maui_sft_shared_kniu_datasets_mlebench_full_jsonl/mle_bench_bashmle_bench_checkpoint_maui_sft_shared_kniu_datasets_mlebench_full_jsonl.jsonl'


In [2]:
df_trj = pd.read_json(file_path, lines=True)


NameError: name 'pd' is not defined

In [ ]:
df_trj.iloc[0]["rollouts"][0]["metrics"]["rollout/duration"]

232.22987089073285

In [ ]:
df_trj.iloc[0]["rollouts"][0]["traj"]["transitions"][-1]["info"]["pred_solution"]

'import os\nimport sys\nimport time\nimport pandas as pd\nimport numpy as np\nfrom sklearn.model_selection import StratifiedKFold\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.metrics import roc_auc_score\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom scipy.sparse import hstack, csr_matrix\n\n# ---------------------------\n# Utility: text preprocessing\n# ---------------------------\n\ndef preprocess_text_series(texts: pd.Series) -> pd.Series:\n    def clean_one(s):\n        if not isinstance(s, str):\n            return ""\n        # Strip surrounding quotes if present\n        if len(s) >= 2 and s[0] == \'"\' and s[-1] == \'"\':\n            s = s[1:-1]\n        # Heuristic: decode common backslash escapes if present\n        try:\n            if ("\\\\x" in s) or ("\\\\u" in s) or ("\\\\n" in s) or ("\\\\t" in s):\n                # encode to bytes then decode unicode_escape to interpret escapes\n                s = s.encode(\'utf-8\', errors

In [ ]:

# Flatten the dataframe
df_flat = flatten_dataframe(df_trj)

print(f"Shape: {df_flat.shape}")
print(f"Columns: {df_flat.columns.tolist()}")
df_flat.head()

Shape: (4800, 9)
Columns: ['task_name', 'task_description', 'code', 'percentile', 'valid_submission', 'eval_error_message', 'eval_duration', 'rollout_duration', 'rollout']


,task_name,task_description,code,percentile,valid_submission,eval_error_message,eval_duration,rollout_duration,rollout
0,detecting-insults-in-social-commentary,# Overview\n\n## Overview\n\n### Prizes\n\n###...,import os\nimport sys\nimport time\nimport pan...,1.000000,True,"Valid submission: percentile=1.0, execution_ti...",6.030760,232.229871,"[{'turn_id': 0, 'action': '', 'observation': '..."
1,leaf-classification,# Overview\n\n## Description\nThere are estima...,#!/usr/bin/env python3\nimport os\nimport sys\...,0.718672,True,Valid submission: percentile=0.718671679197995...,6.031679,239.587325,"[{'turn_id': 0, 'action': '', 'observation': '..."
2,detecting-insults-in-social-commentary,# Overview\n\n## Overview\n\n### Prizes\n\n###...,#!/usr/bin/env python3\nimport os\nimport sys\...,1.000000,True,"Valid submission: percentile=1.0, execution_ti...",5.326396,264.277526,"[{'turn_id': 0, 'action': '', 'observation': '..."
3,nomad2018-predict-transparent-conductors,# Overview\n\n## Description\n\nInnovative mat...,#!/usr/bin/env python3\nimport os\nimport sys\...,0.898180,True,Valid submission: percentile=0.898179749715585...,14.303391,265.535683,"[{'turn_id': 0, 'action': '', 'observation': '..."
4,detecting-insults-in-social-commentary,# Overview\n\n## Overview\n\n### Prizes\n\n###...,#!/usr/bin/env python3\nimport os\nimport sys\...,1.000000,True,"Valid submission: percentile=1.0, execution_ti...",12.928361,280.709331,"[{'turn_id': 0, 'action': '', 'observation': '..."


In [ ]:
save_path = f'/checkpoint/maui_sft/winnieyangwn/amaia_dumps/{run_id}/trajectories/{run_id}_metadata.jsonl'

# Save df_flat to jsonl
df_flat.to_json(save_path, orient='records', lines=True)
print(f"Saved {len(df_flat)} rows to {save_path}")

Saved 4800 rows to /checkpoint/maui_sft/winnieyangwn/amaia_dumps/514/trajectories/514_metadata.jsonl


# Data Structure Description

The context is a list of dictionaries, where each dictionary represents one MLE Bench rollout (an agent's attempt to solve a machine learning task). There are 4,800 rollouts in total.

Each rollout dictionary has the following fields:

1. **"task_name"** (str): The unique identifier for the ML task (e.g., "detecting-insults-in-social-commentary")

2. **"task_description"** (str): Full markdown description of the ML task including overview, evaluation criteria, and submission format

3. **"code"** (str | None): The final Python code solution submitted by the agent

4. **"percentile"** (float | None): Performance percentile (0-100) achieved by the submission. Higher is better. 100 means top performer.

5. **"valid_submission"** (bool | None): Whether the agent produced a valid submission file

6. **"eval_error_message"** (str | None): Evaluation result message - contains success info or error details

7. **"eval_duration"** (float | None): GPU execution time in seconds for evaluation

8. **"rollout_duration"** (float | None): Total time in seconds for the entire rollout

9. **"rollout"** (list[dict]): The full conversation trajectory as a list of turns. Each turn has:
   - "turn_id" (int): Turn number starting from 0
   - "action" (str): The agent's action/response (typically bash commands in XML tags)
   - "observation" (str): The environment's response/prompt to the agent

## Example access patterns:
```python
# Get task name
context[0]["task_name"]

# Get percentile
context[0]["percentile"]

# Get number of turns
len(context[0]["rollout"])

# Get first action
context[0]["rollout"][0]["action"]

# Filter successful submissions
[r for r in context if r["valid_submission"] == True]

# Filter by percentile
[r for r in context if r["percentile"] and r["percentile"] >= 50]
```